In [1]:
import librosa
import IPython.display
import os
import librosa.display
import numpy as np
import random
import matplotlib.pyplot as plt
import datasets

In [2]:
major_dir='/kaggle/input/musical-instrument-chord-classification/Audio_Files/Major'
minor_dir='/kaggle/input/musical-instrument-chord-classification/Audio_Files/Minor'

In [3]:
def select(folder_path):
    files = os.listdir(folder_path)
    files = [f for f in files if os.path.isfile(os.path.join(folder_path, f))]
    file_name=random.choice(files)
    print(file_name)
    return os.path.join(folder_path,file_name)

In [4]:
sample,sr=librosa.load(select(major_dir))

Major_73.wav


In [5]:
def corres_label(sample, sr,clss):
    chroma = librosa.feature.chroma_stft(y=sample, sr=sr)
    chords = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    chord_dict = {chord: [] for chord in chords}
    for i in range(chroma.shape[1]):  
        for j in range(len(chords)):  
            chord_dict[chords[j]].append(chroma[j, i])
    maxi, maxi_class = 0, '#'
    for chord in chord_dict.keys():
        cur_sum = sum(chord_dict[chord])
        if maxi < cur_sum:
            maxi = cur_sum
            maxi_class = chord

    return f'Detected Chord={maxi_class+clss}'

In [6]:
def plot_chromagram(audio_samples, titles, sr=22050, hop_length=512):
    plt.figure(figsize=(25, 50)) 
    
    for i, (sig, title) in enumerate(zip(audio_samples, titles)):
        chroma = librosa.feature.chroma_stft(y=sig, sr=sr, hop_length=hop_length)
        plt.subplot(len(audio_samples), 1, i + 1)
        librosa.display.specshow(chroma, y_axis='chroma', x_axis='time', sr=sr, hop_length=hop_length)
        plt.colorbar(format='%+2.0f dB')
        plt.title(f'Chromagram of {title}')
        plt.tight_layout()

    plt.show()

In [7]:
audio=[]
text=[]
for files in os.listdir(major_dir):
    path=os.path.join(major_dir,files)
    sample,sr=librosa.load(path)
    label=corres_label(sample,sr/2,'Major')
    audio.append(path)
    text.append(label)

In [8]:
for files in os.listdir(minor_dir):
    path=os.path.join(minor_dir,files)
    sample,sr=librosa.load(path)
    label=corres_label(sample,sr/2,'Minor')
    audio.append(path)
    text.append(label)

In [9]:
data = {
    'audio': audio,
    'text': text
}
dataset = datasets.Dataset.from_dict(data)
dataset

Dataset({
    features: ['audio', 'text'],
    num_rows: 859
})

In [10]:
from huggingface_hub import notebook_login
notebook_login()

In [12]:
dataset.push_to_hub('abh1shank/chord_classificarion_audio_data',private=False,commit_message="data_for_chordClassification")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/301 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/abh1shank/chord_classificarion_audio_data/commit/d3bcd9d6f79260b250b22c017ca5fbf62ec76bd8', commit_message='data_for_chordClassification', commit_description='', oid='d3bcd9d6f79260b250b22c017ca5fbf62ec76bd8', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
loaded=datasets.load_dataset('abh1shank/chord_classificarion_audio_data')

Generating train split:   0%|          | 0/859 [00:00<?, ? examples/s]

In [14]:
shuffled_dataset = loaded.shuffle(seed=42)
testing,srr=librosa.load(shuffled_dataset['train']['audio'][7])
corres_label(testing,srr//2,'Major')

'Detected Chord=FMajor'

In [15]:
shuffled_dataset['train']['text'][7]

'Detected Chord=FMajor'